#NIRC2 preprocessing streamlined

Streamlined version of the notebook NIRC2_Preprocessing.

Stable with vip v.0.4.3

##Import <a id='import'></a>

In [ ]:
%reset

In [ ]:
from NIRC2_Preprocessing import *
import numpy as np

from vip.fits import vipDS9
from os.path import join

%matplotlib inline
%load_ext autoreload
%autoreload 2

Open a ds9 window

In [ ]:
ds9 = vipDS9()

---

## Paths

Define paths to RAW images.

In [ ]:
path_flat = '/Users/Olivier/Documents/ULg/VORTEX/Data/RAW/Keck/HR8799/20151024/flat/'
path_sky = '/Users/Olivier/Documents/ULg/VORTEX/Data/RAW/Keck/HR8799/20151024/sky/'
path_sci = '/Users/Olivier/Documents/ULg/VORTEX/Data/RAW/Keck/HR8799/20151024/sci/'

Define paths to PREPROCESSED images if already created.

In [ ]:
path_sky_preprocessed = '/Users/Olivier/Documents/ULg/VORTEX/Data/RAW/Keck/HR8799/20151024/processed/sky_flatted/msky.fits'
path_sci_preprocessed = '/Users/Olivier/Documents/ULg/VORTEX/Data/RAW/Keck/HR8799/20151024/processed/sci_flatted/'

Define the folder path where results are saved (if not required, use None). 

In [ ]:
path_output = '/Users/Olivier/Documents/ULg/VORTEX/Data/RAW/Keck/HR8799/20151024/processed/'

In [ ]:
z = open_fits(join(path_flat,'n0393.fits'))
display_array_ds9(z)

##Preprocessing

If already created, you can load the preprocessed sky and sci images...

In [ ]:
sky_preprocessed = load_images(path_sky_preprocessed)
sci_preprocessed = load_images(path_sci_preprocessed)

... if not, do you want to save the mflat and preprocessed images ?

In [ ]:
save = False

Build the master flat and preprocess the sci and sky images: Let's run !

In [ ]:
# File list
filelist_flat = listing(path_flat, selection=False, ext='fits')
filelist_sci = listing(path_sci, selection=False, ext='fits')
filelist_sky = listing(path_sky, selection=False, ext='fits')

# Let's go
mflat = masterFlat(filelist_flat, save=save, path_output=path_output, verbose=True, filtering=5)
sci_preprocessed = applyFlat(filelist_sci, mflat, save=save, path_output=path_output, verbose=True)
sky_preprocessed = applyFlat(filelist_sky, mflat, save=save, path_output=path_output, verbose=True)

Optional : build the master sky. 

In [ ]:
filename = 'msky'

sky_preprocessed = master(listing(join(path_output,'sky_flatted')), norm=False, save=save, filename=filename, 
                          path_output=join(path_output,'sky_flatted'), verbose=True, filtering=None)

## Vortex position

Display the sci and sky preprocessed images to estimate the position of a dust and the vortex.

In [ ]:
ds9.display(sci_preprocessed,sky_preprocessed, mflat)

Define the estimates for the dust and vortex positions. In addition, we define both the dust and vortex model. 

In [ ]:
dust_center = (599,605)
dust_box_size = 41
dust_fun = gauss2d_sym
dust_fun_parameters = [5]

vortex_center = (527,730)
vortex_box_size = 28
vortex_fun = gauss2d_sym
vortex_fun_parameters = [5]

Let's run ! If **full_output** is True, the vortex_center_from_dust_signature() function will also return:
- The positions of the vortex in the sci images
- The positions of the dust in the sci images
- The positions of the vortex in the sky images
- The positions of the dust in the sky images

In [ ]:
dust_options = {'center':dust_center, 'size':vortex_box_size, 'fun':dust_fun, 'parameters':dust_fun_parameters}
vortex_options = {'center':vortex_center, 'size':vortex_box_size, 'fun':vortex_fun, 'parameters':vortex_fun_parameters}

center_all = vortex_center_from_dust_signature(sci_preprocessed, sky_preprocessed, dust_options, vortex_options, 
                                               verbose=True, full_output=False)

## Registration and cropping

Define the size of the final cube. If **None**, the optimal size is automatically calculated.

In [ ]:
size = None

Let's run !

In [ ]:
cube_reg = cube_registration(sci_preprocessed, center_all, cube_output_size=size, save=True, 
                             verbose=True, path_output=path_output, filename='toto')

ds9.display(sci_preprocessed, cube_reg)

##Bad pixel removal

Under construction ...

##ADI or RDI PCA 

In [ ]:
from vip.pca.pca_local import pca_adi_annular_quad as annular_pca
from vip.pca import pca
from vip.fits import vipDS9, write_fits

Get the parallactic angles.

**NOTE**: If *path_sci_preprocessed* has been used instead of *path_sci*, modify *path_sci* in the argument of **get_parallactic_angles** into *path_sci_preprocessed*.

In [ ]:
angs = get_parallactic_angles(listing(path_sci), save=True, path_output=path_output)

Get the reference cube for RDI

In [ ]:
path_ref_for_RDI = '/Users/Olivier/Documents/ULg/VORTEX/Data/Cube_PSF_PA/Keck/HR8799/20150609/cube_HD219196_20150609.fits'
cube_ref = open_fits(path_ref_for_RDI)

Let's run !

In [ ]:
image = pca(cube_reg, angs, cube_ref=None, ncomp=3, svd_mode='randsvd', full_output=False)
ds9.display(image)

If you want to perform **annular_pca( )** instead of normal **pca()**:

In [ ]:
image = annular_pca(cube_reg, angs, ncomp=5, fwhm=8, svd_mode='randsvd', full_output=False)
ds9.display(image)

Save

In [ ]:
filename = 'final_image.fits'
write_fits(join(path_output,filename), image)